In [37]:
import json
import requests
import pandas as pd

In [38]:
API_ENDPOINT = "http://localhost:3001/api/weaviate"

In [39]:
def get_speaker_embedding(government, meeting_type, year, code, speakerID):
    path = f"data/{government}/{meeting_type}/{year}/turnObjects/{code}.wav.json"
    with open(path, "r") as f:
        data = json.load(f)

    for d in data:
        if d["object"]["speaker"] == speakerID:
            return d["vector"]

    print(f"ERROR! Could not find speakerID in video {code}")

In [40]:
def add_weaviate(government, name, embedding):
    r = requests.post(
        f"{API_ENDPOINT}/insert",
        json={
            "collection": "Speakers",
            "objects": [
                {
                    "object": {"government": government, "name": name},
                    "vector": embedding,
                }
            ],
        },
    )

    if r.status_code != 200:
        print(f"ERROR INSERTING {name}!")
        return
    print(f"Inserted {name} in weaviate")

In [41]:
def is_in_weaviate(embedding):
    r = requests.post(f"{API_ENDPOINT}/searchVector", json={})

    if r.status_code != 200:
        print("ERROR LOOKING FOR SPEAKER!")
        return

    print(r.json())
    # TODO: Return true if found, false otherwise

In [42]:
def handle_file(government, meeting_type, year, path):
    code = path.split("/")[-1].split(".")[0]
    data = pd.read_excel(path)
    done = []
    for index, row in data.iterrows():
        name = row["Naam"]
        speakerID = row["sprekerID"]
        # If speaker is named
        if not pd.isna(name) and speakerID not in done:
            if name.strip().lower() == "inspreker":
                continue
            print(
                f"Row {index+1}: 'Naam' is not empty and its value is '{name}', '{speakerID}', {code}"
            )
            embedding = get_speaker_embedding(government, meeting_type, year, code, speakerID)
            add_weaviate(government, name, embedding)
            done.append(speakerID)


In [43]:
annotated_files_2023 = [
    "/Users/personal/Desktop/scriptie/notebooks/data/hoekschewaard/2023/sheets/1068470.wav.rttm.xlsx",
    "/Users/personal/Desktop/scriptie/notebooks/data/hoekschewaard/2023/sheets/1068534.wav.rttm.xlsx",
    "/Users/personal/Desktop/scriptie/notebooks/data/hoekschewaard/2023/sheets/1068543.wav.rttm.xlsx",
    "/Users/personal/Desktop/scriptie/notebooks/data/hoekschewaard/2023/sheets/1109657.wav.rttm.xlsx",
]
annotated_files_2024 = [
    "/Users/personal/Desktop/scriptie/notebooks/data/hoekschewaard/2024/sheets/1178278.wav.rttm.xlsx",
    "/Users/personal/Desktop/scriptie/notebooks/data/hoekschewaard/2024/sheets/1178261.wav.rttm.xlsx",
    "/Users/personal/Desktop/scriptie/notebooks/data/hoekschewaard/2024/sheets/1192781.wav.rttm.xlsx",
]
for path in annotated_files_2023:
    handle_file("hoekschewaard", "vergadering", "2023", path)

Row 3: 'Naam' is not empty and its value is 'Inspreker ', 'SPEAKER_01', 1068470


FileNotFoundError: [Errno 2] No such file or directory: 'data/hoekschewaard/vergadering/2023/turnObjects/1068470.wav.json'